<a href="https://colab.research.google.com/github/omverkill/Comsys-Hackathon_Emirates/blob/main/genderModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Unzip dataset into /content
!unzip -q '/content/drive/MyDrive/Cosmys/Comys_Hackathon5 new.zip' -d /content/gender_data


Mounted at /content/drive


In [4]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score


In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)), # Resize images to a consistent size (e.g., 224x224)
        transforms.RandomHorizontalFlip(),     # optional: add minor augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3) # normalize to [-1,1]
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)), # Resize validation images as well
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ]),
}


In [6]:
data_dir = '/content/gender_data/Comys_Hackathon5/Task_A'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2)
               for x in ['train', 'val']}
class_names = image_datasets['train'].classes

##After Preprocessing

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Replace classifier for binary classification
model.classifier[1] = nn.Linear(model.last_channel, 2)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 82.1MB/s]


##Loss and Optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)


##Training Loop

In [9]:
def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])
            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")

            if phase == 'val':
                precision = precision_score(all_labels, all_preds, average='binary')
                recall = recall_score(all_labels, all_preds, average='binary')
                f1 = f1_score(all_labels, all_preds, average='binary')
                print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

    return model

model = train_model(model, criterion, optimizer, num_epochs=5)


Epoch 1/5
Train Loss: 0.3028, Acc: 0.8759
Val Loss: 0.3222, Acc: 0.8673
Precision: 0.8537, Recall: 0.9937, F1-Score: 0.9184

Epoch 2/5
Train Loss: 0.2175, Acc: 0.9190
Val Loss: 0.2546, Acc: 0.8910
Precision: 0.9119, Recall: 0.9464, F1-Score: 0.9288

Epoch 3/5
Train Loss: 0.2080, Acc: 0.9190
Val Loss: 0.2599, Acc: 0.8768
Precision: 0.8841, Recall: 0.9621, F1-Score: 0.9215

Epoch 4/5
Train Loss: 0.1851, Acc: 0.9304
Val Loss: 0.2626, Acc: 0.8839
Precision: 0.8873, Recall: 0.9685, F1-Score: 0.9261

Epoch 5/5
Train Loss: 0.1809, Acc: 0.9216
Val Loss: 0.2966, Acc: 0.8863
Precision: 0.8768, Recall: 0.9874, F1-Score: 0.9288


In [10]:
torch.save(model.state_dict(), 'gender_classifier.pth')
